In [1]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from tensorflow.keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path
from keras.models import Model
import shutil

Using TensorFlow backend.


In [2]:
batch_size = 32
img_height = 224
img_width = 224
NUM_CLASSES = 102
epochs = 100

In [3]:
print(tf.__version__)

2.3.0


In [4]:
tf.keras.preprocessing.image.ImageDataGenerator

tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [5]:
orig_dataset_dir = 'E:/Github/thesis/flowerDataset/Flower/'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
new_flower_dataset_dir = 'E:/Github/thesis/flowerDataset/new_flower_dataset/'

In [6]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)

In [7]:
df = pd.DataFrame(data_labels)

In [8]:
labels = df[1][3][0] #loads the labels that is stored in dataframe format

In [9]:
def load_images_from_folder(orig_dataset_dir):
    images = []
    for i, filename in enumerate(os.listdir(orig_dataset_dir)):
        #check label[i] value
        label = str(labels[i])        
        dst_dir = new_flower_dataset_dir+label
        if os.path.isdir(dst_dir): #check if director exist the copy file
            shutil.copy(orig_dataset_dir+filename, dst_dir)
        else: #create folder with name label and copy over file 
            os.makedirs(dst_dir)
            shutil.copy(orig_dataset_dir+filename, dst_dir)
            
        #if label value folder exists? if not create it else copy image[filename] to new directory
        

In [10]:
#load_images_from_folder(orig_dataset_dir)

In [19]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    new_flower_dataset_dir,
    labels="inferred",
    label_mode="categorical",  # categorical, binary
    # class_names=['0', '1', '2', '3', ...]
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training",
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    new_flower_dataset_dir,
    labels="inferred",
    label_mode="categorical",  # categorical, binary
    # class_names=['0', '1', '2', '3', ...]
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation",
)



Found 8189 files belonging to 102 classes.
Using 7371 files for training.
Found 8189 files belonging to 102 classes.
Using 1637 files for validation.


In [18]:
ds_train.list_files("image*")

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: image*'

In [12]:
"""def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y


ds_train = ds_train.map(augment)"""


'def augment(x, y):\n    image = tf.image.random_brightness(x, max_delta=0.05)\n    return image, y\n\n\nds_train = ds_train.map(augment)'

In [13]:
#https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import keras
import numpy as np

In [14]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(NUM_CLASSES, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [16]:
input_shape = (224, 224, 3)
optim_1 = keras.optimizers.Adam(learning_rate=0.001)

#n_steps = ds_train.samples // BATCH_SIZE
#n_val_steps = ds_validation.samples // BATCH_SIZE

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, NUM_CLASSES, optim_1, fine_tune=0)

In [ ]:
vgg_history = vgg_model.fit(ds_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=ds_validation,
                            verbose=2)

Epoch 1/100
231/231 - 18s - loss: 4.6118 - accuracy: 0.0301 - val_loss: 4.6008 - val_accuracy: 0.0367
Epoch 2/100
231/231 - 18s - loss: 4.5875 - accuracy: 0.0309 - val_loss: 4.5818 - val_accuracy: 0.0367
Epoch 3/100
231/231 - 17s - loss: 4.5681 - accuracy: 0.0309 - val_loss: 4.5670 - val_accuracy: 0.0367
Epoch 4/100
231/231 - 17s - loss: 4.5525 - accuracy: 0.0309 - val_loss: 4.5554 - val_accuracy: 0.0367
Epoch 5/100
231/231 - 17s - loss: 4.5401 - accuracy: 0.0309 - val_loss: 4.5465 - val_accuracy: 0.0367
Epoch 6/100
231/231 - 17s - loss: 4.5304 - accuracy: 0.0309 - val_loss: 4.5396 - val_accuracy: 0.0367
Epoch 7/100
231/231 - 17s - loss: 4.5227 - accuracy: 0.0309 - val_loss: 4.5345 - val_accuracy: 0.0367
Epoch 8/100
231/231 - 16s - loss: 4.5169 - accuracy: 0.0309 - val_loss: 4.5307 - val_accuracy: 0.0367
Epoch 9/100
231/231 - 17s - loss: 4.5124 - accuracy: 0.0309 - val_loss: 4.5279 - val_accuracy: 0.0367
Epoch 10/100
231/231 - 17s - loss: 4.5090 - accuracy: 0.0301 - val_loss: 4.5260 - 

In [ ]:
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)